# 环境配置（此处使用AutoDL线上平台）
Cuda    12.1

GPU     RTX 4090D(24GB) * 1

CPU     15 vCPU Intel(R) Xeon(R) Platinum 8474C

Python            3.10(ubuntu22.04)

PyTorch           2.1.0

transformers      4.41.2

seqeval           1.2.2

In [11]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizerFast, BertModel, AdamW
from seqeval.metrics import classification_report

In [12]:
# 参数设置
BATCH_SIZE = 16  # 每个 batch 包含 N 个样本
EPOCHS = 3  # 训练 3 个 epoch
LEARNING_RATE = 5e-5  # 学习率

In [13]:
# 标签定义
LABELS = [
    "O",
    "B-NAME", "I-NAME", "B-CONT", "I-CONT", "B-EDU", "I-EDU", "B-TITLE", "I-TITLE", "B-ORG", "I-ORG", "B-RACE", "I-RACE", "B-PRO", "I-PRO", "B-LOC", "I-LOC"
]
label2id = {label: i for i, label in enumerate(LABELS)}  # 标签到 ID 的映射
id2label = {i: label for i, label in enumerate(LABELS)}  # ID 到标签的映射
print('标签种类：', len(LABELS))
print(label2id)
print(id2label)

标签种类： 17
{'O': 0, 'B-NAME': 1, 'I-NAME': 2, 'B-CONT': 3, 'I-CONT': 4, 'B-EDU': 5, 'I-EDU': 6, 'B-TITLE': 7, 'I-TITLE': 8, 'B-ORG': 9, 'I-ORG': 10, 'B-RACE': 11, 'I-RACE': 12, 'B-PRO': 13, 'I-PRO': 14, 'B-LOC': 15, 'I-LOC': 16}
{0: 'O', 1: 'B-NAME', 2: 'I-NAME', 3: 'B-CONT', 4: 'I-CONT', 5: 'B-EDU', 6: 'I-EDU', 7: 'B-TITLE', 8: 'I-TITLE', 9: 'B-ORG', 10: 'I-ORG', 11: 'B-RACE', 12: 'I-RACE', 13: 'B-PRO', 14: 'I-PRO', 15: 'B-LOC', 16: 'I-LOC'}


In [14]:
# 工具函数，读取数据，并按句子切分
def read_data(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()  # 读取文件中的所有行

    texts, labels = [], []
    text, label = [], []
    for line in lines:
        if line.strip() == "": # 句子切分处，这里检测是否是空行，我们的语料中使用空行切割句子
            if text:
                texts.append(text)  # 将一个句子的文本添加到文本列表中
                # 将一个句子的标签转换为 ID ，添加到标签列表中
                labels.append([label2id[l] for l in label])
                text, label = [], []
        else: # 非句子切分处，也就是如同“项 B-TITLE”这种形式的行
            token, tag = line.strip().split()  # 分割文本和标签
            text.append(token)
            label.append(tag)

    if text:
        texts.append(text)
        labels.append([label2id[l] for l in label])

    return texts, labels  # 返回文本和标签列表，这两个都是二维数组
# 读取数据
train_texts, train_labels = read_data("train.txt")
test_texts, test_labels = read_data("test.txt")

print('训练集句子数量：', len(train_texts), train_texts[0])
print('训练集句子标签：', len(train_labels), train_labels[0])
print('测试集句子数量：', len(test_texts), test_texts[0])
print('测试集句子标签：', len(test_labels), test_labels[0])

训练集句子数量： 3 ['高', '勇', '：', '男', '，', '中', '国', '国', '籍', '，', '无', '境', '外', '居', '留', '权', '，']
训练集句子标签： 3 [1, 2, 0, 0, 0, 3, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0]
测试集句子数量： 2 ['常', '建', '良', '，', '男', '，']
测试集句子标签： 2 [1, 2, 2, 0, 0, 0]


In [15]:
# 管理数据
class NERDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)  # 返回数据集大小

    def __getitem__(self, idx):
        # 这里就是按一个句子，一个句子去处理的
        text = self.texts[idx]  # 获取第 idx 个样本（句子）的文本，① 这是一个句子的文本
        labels = self.labels[idx]  # 获取第 idx 个样本（句子）的标签，这是一个句子的BIO标签，② 但是使用数字形式标注

        # 转换成bert的token格式
        encoding = self.tokenizer(
            text,
            is_split_into_words=True,
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors="pt"
        )
        # print(111, text)
        # 111 ['高', '勇', '：', '男', '，', '中', '国', '国', '籍', '，', '无', '境', '外', '居', '留', '权', '，']
        
        # print(222, labels)
        # 222 [1, 2, 0, 0, 0, 3, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0]
        
        # print(333, encoding)
        # 该对象有3个属性，但后面会被补充为4个属性
        
        # input_ids：注意是二维数组，101是<cls>特殊标记，然后不足512，padding到512
        # input_ids：tensor([[ 101, 7770, 1235, 8038, 4511, 8024,  704, 1744, 1744, 5093, 8024, 3187, 1862, 1912, 2233, 4522, 3326, 8024,  102,    0,    0,    0,    0,    0...]])
        
        # token_type_ids：用于标记两个句子，比如一个问题标记为0，答案标记为1
        # token_type_ids：tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...]])

        # attention_mask：设置是否参与注意力机制，cls标记为1，但pad标记为0
        # attention_mask：tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0...
        
        word_ids = encoding.word_ids()  # 获取每个 token 所对应的单词索引
        label_ids = []
        # previous_word_idx = None
        # 关于word_ids
        # print(444, word_ids)
        # 也是512位，[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, None, None, None, None, None, None, None, ....]
        # 作用：如果某个子词是原始句子中的一个token，它将返回该单词在原始文本中的索引。
        # 而如果子词是由于tokenization过程产生的，则不能直接映射到原始单词（比如，是因为标点符号、特殊字符或单词被切分产生的子词），它会返回None。
        # 列表的长度与input_ids相同，意味着每个tokenized的子词都有一个对应的word_id。

        # 下面主要用于生成第四种数据，更改后的label数据（前面的三种数据分别是：text、labels、encoding），主要是将原来的label拓展到512维
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # None位置填充位置的标签设为-100，仅仅是一个标记，不要和正式的标签id重合
            else:
                label_ids.append(labels[word_idx])
            # 下面的代码有些冗余
            # elif word_idx != previous_word_idx:
            #     label_ids.append(labels[word_idx])  # 第一次出现的单词的标签
            # else:
            #     label_ids.append(
            #         labels[word_idx] if word_idx is not None else -100)
            # previous_word_idx = word_idx
        encoding['labels'] = torch.tensor(label_ids)  # 将标签添加到encoding中，这下encoding就有4个属性了：input_ids、token_type_ids、attention_mask、labels
        # print(444, encoding['labels'])
        # None的位置都被标记为了-100
        # 444 tensor([-100, 1, 2, 0, 0, 0, 3, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, -100, -100, -100, -100, -100, -100,
       
        # 返回encoding
        # 理解下squeeze：torch.tensor([[1,2,3]]).squeeze(0)，返回tensor([1, 2, 3])，从二维压缩到了一维
        # Transformers库的tokenizer，用于存储文本处理后的各种信息，如input_ids、attention_mask等。这些张量可能具有一个额外的维度，通常是批处理维度（即使只有一个样本，也会默认添加一个维度，形状为(1, ...))。
        # squeeze(0)：这是PyTorch中的一个操作，用于移除张量中大小为1的维度。在这里，0指定要移除第一个维度。如果val是一个形状为(1, ...,)的张量，val.squeeze(0)就会返回一个形状为(...)的张量，即去掉了那个单元素的批次维度，使得处理单个样本时更加方便。

        return {key: val.squeeze(0) for key, val in encoding.items()}


In [16]:
# 创建模型
class BertForNER(nn.Module):
    def __init__(self, num_labels):
        super(BertForNER, self).__init__()
        self.bert = BertModel.from_pretrained(
            "bert-base-chinese")  # 加载预训练的 BERT 模型
        self.dropout = nn.Dropout(0.3)  # 添加 dropout 层
        self.classifier = nn.Linear(
            self.bert.config.hidden_size, num_labels)  # 线性层，承当分类任务，在这里该线性层的输入是768维的，输出是17维的，也就是做17分类
        # print(111, self.bert.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask=None, labels=None):
        # 这三个输入的大小都是[2, 512], 也就是输入两个句子，每个句子包含512个单词
        # 这两个句子是我划分的batch大小, 因为是在本地尝试运行，所以划分的比较小，一般来说可能是8, 16或者32
        # print(1, input_ids, input_ids.shape)
        # 1 tensor([[ 101,  122,  130,  ...,    0,    0,    0], [ 101, 7770, 1235,  ...,    0,    0,    0]]) torch.Size([2, 512])
        
        # print(2, attention_mask, attention_mask.shape)
        # 2 tensor([[1, 1, 1,  ..., 0, 0, 0], [1, 1, 1,  ..., 0, 0, 0]]) torch.Size([2, 512])

        # print(3, labels, labels.shape)
        # 3 tensor([[-100,    0,    0,  ..., -100, -100, -100], [-100,    1,    2,  ..., -100, -100, -100]]) torch.Size([2, 512])
        
        outputs = self.bert(
            input_ids, attention_mask=attention_mask)  # 获取 BERT 的输出
        sequence_output = outputs.last_hidden_state
        sequence_output = self.dropout(sequence_output)  # 应用 dropout
        logits = self.classifier(sequence_output)  # 获取分类结果
        # print(111, logits, logits.shape)
        # 这里的输出大小是, [2, 512, 17], 也就是对两个句子, 每个句子都包含512个单词，对每个单词都进行17分类
        # 111 tensor([[[-0.9038, -0.6527, -1.1142,  ..., -0.0950,  0.4526,  0.4314],
        #    [-0.2693, -0.2534, -0.9892,  ..., -0.2062,  0.5349,  0.3265],
        #    [ 0.1514,  0.0874, -0.6728,  ...,  0.1152, -0.0778, -0.0510],
        #    ...,
        #    [ 0.2086, -0.7756, -0.4723,  ...,  0.1401, -0.9681,  1.4077],
        #    [-0.0785, -1.0320, -0.4171,  ...,  0.0846, -0.6143,  1.1130],
        #    [ 0.4348, -0.6476, -0.0975,  ...,  0.8670, -0.5920,  1.0222]],

        #   [[-0.2587,  0.4648, -0.4417,  ...,  0.1298,  0.5619, -0.0681],
        #    [ 0.7163, -0.7920, -0.8147,  ..., -0.3541, -0.7456,  0.7558],
        #    [ 0.5359, -0.0739, -0.2890,  ..., -0.3686, -0.3514, -0.1295],
        #    ...,
        #    [ 0.3715, -0.9656, -0.4320,  ...,  0.2127, -0.6772,  0.7296],
        #    [ 0.6422, -1.1092, -0.3876,  ...,  0.2827, -0.6668,  1.0752],
        #    [ 0.0343, -0.8732, -0.6937,  ...,  0.5872, -0.6445,  0.7841]]],
        #  grad_fn=<ViewBackward0>) torch.Size([2, 512, 17])
        
        loss = None
        if labels is not None: # 如果提供了正确标签，则我们就可以对比预测标签和正确标签，从而计算交叉熵损失。如果没有提供标签，则不计算交叉熵损失
            loss_fn = nn.CrossEntropyLoss()  # 定义损失函数
            # 注释：attention_mask.view(-1) 是一个展平操作，
            active_loss = attention_mask.view(-1) == 1  # 只计算attentionmask=1的标签的损失，⚠️但是这里岂不是<cls>也参与损失计算
            # print(222, active_loss, active_loss.shape)
            # 222 tensor([ True,  True,  True,  ..., False, False, False]) torch.Size([1024]), 这里是将两个512单词的句子，展开成了一个1024单词的序列，也就是展平了

            active_logits = logits.view(-1, logits.shape[-1])
            # print(333, active_logits, active_logits.shape)
            # 这里获取的是1024个单词，每个单词的17分类结果
            # 333 tensor([[-0.9038, -0.6527, -1.1142,  ..., -0.0950,  0.4526,  0.4314],
            #         [-0.2693, -0.2534, -0.9892,  ..., -0.2062,  0.5349,  0.3265],
            # ...
            #         [ 0.0343, -0.8732, -0.6937,  ...,  0.5872, -0.6445,  0.7841]],
            #        grad_fn=<ViewBackward0>) torch.Size([1024, 17])

            # torch.where函数接受三个参数：条件、满足条件时的值、不满足条件时的值。它会根据提供的条件张量（第一个参数）来选择第二个或第三个参数中的值，形成一个新的张量。
            # active_loss：这是一个布尔张量，表示哪些位置的token是有效（需要计算损失）的，哪些是无效的（如padding，不需要计算损失）。比如，[True, True, False, True, False]意味着前两个和第四个位置的token是有效数据。
            # labels.view(-1)：将原始的标签张量labels展平成一维张量，方便与active_loss进行一一对应。比如，原始的labels可能是(batch_size, max_seq_length)形状，展平后变成一个长列表。
            # loss_fn.ignore_index：这是损失函数（如CrossEntropyLoss）的一个属性，指定了一个特殊的值，当标签等于这个值时，对应的样本在损失计算中会被忽略。常用值是-100，但可以根据具体损失函数的配置而变化。
            # 这里打印了一下, loss_fn.inore_index是-100
            # 感觉这一步稍微有一些多余，因为前面已经在label中按位置填充-100了，⚠️代码需要再优化一下
            active_labels = torch.where(
                active_loss,
                labels.view(-1),
                torch.tensor(loss_fn.ignore_index).type_as(labels)
            )
            # print(444, active_labels, active_labels.shape)
            # 444 tensor([-100,    0,    0,  ..., -100, -100, -100]) torch.Size([1024])

            # active_logits就是预测值，active_labels就是真实值
            loss = loss_fn(active_logits, active_labels)  # 计算损失
            # print(555, loss, loss.shape)
            # 555 tensor(2.8228, grad_fn=<NllLossBackward0>) torch.Size([])
        return loss, logits  # 返回损失和分类结果

In [17]:
# 加载预训练的BERT模型和分词器
tokenizer = BertTokenizerFast.from_pretrained("bert-base-chinese")

In [18]:
# 测试用的数据，仅用于查看输出
# NERDataset(train_texts, train_labels, tokenizer)[0]

In [19]:
# 数据加载
train_dataset = NERDataset(train_texts, train_labels, tokenizer)
test_dataset = NERDataset(test_texts, test_labels, tokenizer)

print('训练集句子数量：', len(train_dataset))
print('训练集句子数量：', train_dataset[0])

训练集句子数量： 3
训练集句子数量： {'input_ids': tensor([ 101, 7770, 1235, 8038, 4511, 8024,  704, 1744, 1744, 5093, 8024, 3187,
        1862, 1912, 2233, 4522, 3326, 8024,  102,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
      

In [20]:
# 创建模型
model = BertForNER(num_labels=len(LABELS))  # 初始化模型

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [21]:
# 切分数据，使用dataloader
train_dataloader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True)  # 训练数据加载器
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE)  # 测试数据加载器

In [22]:
# 优化器
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)  # AdamW 优化器

d:\softwares\python\lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [23]:
# 训练函数
def train(device):
    model.train()  # 设置模型为训练模式
    model.to(device)  # 将模型移动到指定设备
    for epoch in range(EPOCHS):
        total_loss = 0
        for batch_idx, batch in enumerate(train_dataloader):
            optimizer.zero_grad()  # 清空梯度
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            loss, _ = model(
                input_ids, attention_mask=attention_mask, labels=labels)
            loss.backward()  # 反向传播
            optimizer.step()  # 更新参数
            total_loss += loss.item()
            if (batch_idx + 1) % 1 == 0:  # 每 10 个 batch 打印一次进度
                print(
                    f"Epoch {epoch + 1}/{EPOCHS}, Batch {batch_idx + 1}/{len(train_dataloader)}, Loss: {loss.item()}")
        avg_loss = total_loss / len(train_dataloader)
        print(f"Epoch {epoch + 1}/{EPOCHS}, Average Loss: {avg_loss}")

In [28]:

# 评估函数
def evaluate(device):
    model.eval()  # 设置模型为评估模式
    model.to(device)  # 将模型移动到指定设备
    true_labels = []
    pred_labels = []
    for batch_idx, batch in enumerate(test_dataloader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        with torch.no_grad():
            _, logits = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(logits, dim=-1)

        # 只考虑有效的标签
        true_labels_batch = [[id2label[l] for l in label if l != -100]
                             for label in labels.cpu().numpy()]
        pred_labels_batch = [[id2label[p] for p, l in zip(
            pred, label) if l != -100] for pred, label in zip(predictions.cpu().numpy(), labels.cpu().numpy())]
        true_labels.extend(true_labels_batch)
        pred_labels.extend(pred_labels_batch)

        print(111, true_labels_batch)
        print(222, pred_labels_batch)

        if (batch_idx + 1) % 10 == 0:  # 每 10 个 batch 打印一次进度
            print(f"Evaluating Batch {batch_idx + 1}/{len(test_dataloader)}")

    # 使用 seqeval 计算评价指标
    print("Evaluation Results:")
    print(classification_report(true_labels, pred_labels))


In [25]:
# 预测函数
def predict(text, device):
    model.eval()  # 设置模型为评估模式
    model.to(device)  # 将模型移动到指定设备
    tokens = tokenizer.tokenize(text)  # 对输入文本进行分词，这里是先拆分成单个汉字，单个汉字的形式，和我们的train语料保持一致
    # print(111, text)
    # print(222, tokens)
    encoding = tokenizer(
        tokens,
        is_split_into_words=True,
        return_tensors="pt",
        truncation=True,
        padding='max_length',
        max_length=512
    )
    # print(333, encoding)

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        _, logits = model(input_ids, attention_mask=attention_mask)
    predictions = torch.argmax(
        logits, dim=-1).squeeze().cpu().numpy()  # 获取预测结果
    # print(444, predictions)

    predicted_labels = [id2label[p] for p in predictions]
    # print(555, predicted_labels)
    return list(zip(tokens, predicted_labels[1:len(tokens) + 1])) # 这个1是去除cls位置的预测

In [26]:
# 训练
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train(device)

Epoch 1/3, Batch 1/1, Loss: 3.061004400253296
Epoch 1/3, Average Loss: 3.061004400253296
Epoch 2/3, Batch 1/1, Loss: 2.1669387817382812
Epoch 2/3, Average Loss: 2.1669387817382812
Epoch 3/3, Batch 1/1, Loss: 1.5948504209518433
Epoch 3/3, Average Loss: 1.5948504209518433


In [29]:
# 评估
evaluate(device)

111 [['B-NAME', 'I-NAME', 'I-NAME', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PRO', 'I-PRO', 'B-EDU', 'I-EDU', 'O', 'B-TITLE', 'I-TITLE', 'I-TITLE', 'I-TITLE', 'I-TITLE']] [['B-NAME', 'I-NAME', 'I-NAME', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PRO', 'I-PRO', 'B-EDU', 'I-EDU', 'O', 'B-TITLE', 'I-TITLE', 'I-TITLE', 'I-TITLE', 'I-TITLE']]
222 [['O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-TITLE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-TITLE', 'O']] [['O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-TITLE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-TITLE', 'O']]
Evaluation Results:
              precision    recall  f1-score   support

         EDU       0.00      0.00      0.00         1
        NAME       0.00      0.00      0.00         1
         PRO       0.00      0.00      0.00         1
       TITLE       0.00      0.00      0.00         1

   micro avg       0.00      0.00      0.00         4


d:\softwares\python\lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# 预测
sample_text = "韦棣华女士曾任教于文华书院"
prediction = predict(sample_text, device)
print(prediction)

111 韦棣华女士曾任教于文华书院
222 ['韦', '棣', '华', '女', '士', '曾', '任', '教', '于', '文', '华', '书', '院']
333 {'input_ids': tensor([[ 101, 7504, 3479, 1290, 1957, 1894, 3295,  818, 3136,  754, 3152, 1290,
          741, 7368,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
   